In [1]:
import tensorflow as tf

/Users/seongjuhyeon/workspace/tensorflowZeroToAll/venv/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
# To solve downloading files.
## https://github.com/tensorflow/tensorflow/issues/10779
## run the /Applications/Python 3.6/Install Certificates.command bash script to install newer certs.
mnist = input_data.read_data_sets("../MNIST_data/", one_hot=True)
print(mnist.train.num_examples)

Extracting ../MNIST_data/train-images-idx3-ubyte.gz
Extracting ../MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../MNIST_data/t10k-labels-idx1-ubyte.gz
55000


In [4]:
# 0 ~ 9까지의 숫자 갯수
nb_classes = 10

In [5]:
# MNIST Data image of shape = 28 * 28 = 784
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, nb_classes]) # one-hot으로 주어짐.

### lab10-3-xavier-for-MNIST에서 shape를 392 -> 650, depth를 4 -> 7로만 증가시켜봄.
학습결과 accuracy가 더 잘 나오는지 확인해보기

In [6]:
W1 = tf.get_variable("W1", shape=[784, 600],
                     initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.get_variable(name="b1",
                     shape=[600])
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

In [7]:
W2 = tf.get_variable("W2", shape=[600, 600],
                     initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.get_variable(name="b2",
                     shape=[600])
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

In [8]:
W3 = tf.get_variable("W3", shape=[600, 600],
                     initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.get_variable(name="b3",
                     shape=[600])
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

In [9]:
W4 = tf.get_variable("W4", shape=[600, 600],
                     initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.get_variable(name="b4",
                     shape=[600])
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

In [10]:
W5 = tf.get_variable("W5", shape=[600, 600],
                     initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.get_variable(name="b5",
                     shape=[600])
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

In [11]:
W6 = tf.get_variable("W6", shape=[600, 600],
                     initializer=tf.contrib.layers.xavier_initializer())
b6 = tf.get_variable(name="b6",
                     shape=[600])
L6 = tf.nn.relu(tf.matmul(L5, W6) + b6)

In [12]:
W7 = tf.get_variable("W7", shape=[600, nb_classes],
                     initializer=tf.contrib.layers.xavier_initializer())
b7 = tf.get_variable(name="b7",
                     shape=[nb_classes])
hypothesis = tf.matmul(L6, W7) + b7

In [13]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=hypothesis, labels=Y))

In [14]:
# Minimize cost
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

In [15]:
# Correct Prediction
prediction = tf.argmax(hypothesis, 1)
real = tf.argmax(Y, 1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction, real), dtype=tf.float32))

In [16]:
# Prepare session
session = tf.Session()
session.run(tf.global_variables_initializer())

In [17]:
# Slice the data, epochs, batch_size
training_epochs = 15
batch_size = 100
total_iterator = int(mnist.train.num_examples / batch_size)

print(mnist.train.num_examples)
print(total_iterator)

55000
550


In [18]:
# Fit the train
for epoch in range(training_epochs):
    avg_cost = 0
    total_iterator = total_iterator
    
    for i in range(total_iterator):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c_val, _ = session.run([cost, train], feed_dict=feed_dict)
        avg_cost = c_val / batch_size
    
    print("Epoch:", "%04d" % (epoch+1), ", Cost:", "{:.5f}".format(avg_cost))

print('Learning Finished!')

print("Accuracy:", session.run(accuracy,
                               feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Epoch: 0001 , Cost: 0.00278
Epoch: 0002 , Cost: 0.00373
Epoch: 0003 , Cost: 0.00221
Epoch: 0004 , Cost: 0.00202
Epoch: 0005 , Cost: 0.00198
Epoch: 0006 , Cost: 0.00050
Epoch: 0007 , Cost: 0.00138
Epoch: 0008 , Cost: 0.00182
Epoch: 0009 , Cost: 0.00108
Epoch: 0010 , Cost: 0.00349
Epoch: 0011 , Cost: 0.00231
Epoch: 0012 , Cost: 0.00190
Epoch: 0013 , Cost: 0.00180
Epoch: 0014 , Cost: 0.00222
Epoch: 0015 , Cost: 0.00154
Learning Finished!
Accuracy: 0.946


### 결과가 deep & wide nn으로 했는데, 오히려 10-3보다 결과(0.9718 -> 0.946)가 안 좋고, Cost가 작아지다가 커지기도 함.
이유는 아마도 over fitting이 됬을 가능성이 있음. 너무 해당 테스트 데이터에 최적화되었을 가능성이 있음.
이를 해결해야 함.
방법은 dropout 이 있음.